In [8]:
! python --version

Python 3.11.11


In [7]:
# --------------------------------------------
# 1) Installation d'Experta dans le notebook
# --------------------------------------------
! pip uninstall frozendict experta --yes
! pip install frozendict>=2.3.4   # on installe manuellement la version la plus récente
! pip install experta --no-deps   # on installe Experta sans vérifier ses deps (qui pin frozendict==1.2)

Found existing installation: frozendict 2.4.6
Uninstalling frozendict-2.4.6:
  Successfully uninstalled frozendict-2.4.6
Found existing installation: experta 1.9.4
Uninstalling experta-1.9.4:
  Successfully uninstalled experta-1.9.4
  Using cached experta-1.9.4-py3-none-any.whl.metadata (5.0 kB)
Using cached experta-1.9.4-py3-none-any.whl (35 kB)


In [1]:
# --------------------------------------------
# 2) Import des modules
# --------------------------------------------
from experta import *

# --------------------------------------------
# 3) Définition des classes de faits
# --------------------------------------------
class Parent(Fact):
    """ Un fait Parent(p=..., c=...) signifie que p est parent de c. """
    pass

class Ancestor(Fact):
    """ Un fait Ancestor(p=..., c=...) signifie que p est ancêtre de c. """
    pass

class Grandparent(Fact):
    """ Un fait Grandparent(p=..., c=...) signifie que p est grand-parent de c. """
    pass

In [2]:
# --------------------------------------------
# 4) Définition du moteur de règles
# --------------------------------------------
class FamilyEngine(KnowledgeEngine):
    """
    Moteur de règles pour déduire:
      - Ancestor (par la règle directe + transitivité)
      - Grandparent
    """

    # Règle 1: Si X est parent de Y => X est ancêtre de Y
    @Rule(Parent(p=MATCH.p, c=MATCH.c))
    def direct_ancestor(self, p, c):
        self.declare(Ancestor(p=p, c=c))

    # Règle 2: Si X est ancêtre de Y et Y est parent de Z => X est ancêtre de Z
    @Rule(
        Ancestor(p=MATCH.x, c=MATCH.y),
        Parent(p=MATCH.y, c=MATCH.z)
    )
    def transitive_ancestor(self, x, y, z):
        self.declare(Ancestor(p=x, c=z))

    # Règle 3: Si X est parent de Y et Y est parent de Z => X est grand-parent de Z
    @Rule(
        Parent(p=MATCH.x, c=MATCH.y),
        Parent(p=MATCH.y, c=MATCH.z)
    )
    def deduce_grandparent(self, x, y, z):
        self.declare(Grandparent(p=x, c=z))

In [3]:
# --------------------------------------------
# 5) Exécution du moteur de règles
# --------------------------------------------
# Création de l'instance du moteur
engine = FamilyEngine()
engine.reset()

# Déclaration des faits de départ
engine.declare(Parent(p="jean",    c="marie"))
engine.declare(Parent(p="marie",   c="suzanne"))
engine.declare(Parent(p="suzanne", c="thomas"))

# Exécution du chaînage avant
engine.run()

In [5]:
engine.facts

FactList([(0, InitialFact()),
          (1, Parent(p='jean', c='marie')),
          (2, Parent(p='marie', c='suzanne')),
          (3, Parent(p='suzanne', c='thomas')),
          (4, Grandparent(p='marie', c='thomas')),
          (5, Ancestor(p='suzanne', c='thomas')),
          (6, Grandparent(p='jean', c='suzanne')),
          (7, Ancestor(p='marie', c='suzanne')),
          (8, Ancestor(p='marie', c='thomas')),
          (9, Ancestor(p='jean', c='marie')),
          (10, Ancestor(p='jean', c='suzanne')),
          (11, Ancestor(p='jean', c='thomas'))])

In [6]:
# --------------------------------------------
# 6) Récupération et interrogation des faits
# --------------------------------------------
all_facts = list(engine.facts.values())  # engine.facts est un dict {id: Fact}

def find_children_of(parent_name):
    """
    Renvoie la liste des enfants de 'parent_name' selon Parent(p=..., c=...).
    """
    return [f["c"] for f in all_facts
            if isinstance(f, Parent) and f["p"] == parent_name]

def find_grandparents_of(child_name):
    """
    Renvoie la liste des grands-parents de 'child_name' selon Grandparent(p=..., c=...).
    """
    return [f["p"] for f in all_facts
            if isinstance(f, Grandparent) and f["c"] == child_name]

def is_grandparent(gp_name, child_name):
    """
    Retourne True si un fait Grandparent(gp_name, child_name) existe.
    """
    return any(isinstance(f, Grandparent) and f["p"] == gp_name and f["c"] == child_name
               for f in all_facts)

def find_ancestors_of(child_name):
    """
    Renvoie la liste des ancêtres de 'child_name' selon Ancestor(p=..., c=...).
    """
    return [f["p"] for f in all_facts
            if isinstance(f, Ancestor) and f["c"] == child_name]

# --------------------------------------------
# 7) Répondre aux questions
# --------------------------------------------

# Q1. Qui est le grand-parent de thomas ?
gp_of_thomas = find_grandparents_of("thomas")
print("Q1. Grand-parent(s) de thomas :", gp_of_thomas)
# => Attendu: ['marie']

# Q2. Qui est l’enfant de jean ?
children_of_jean = find_children_of("jean")
print("Q2. Enfant(s) de jean :", children_of_jean)
# => Attendu: ['marie']

# Q3. Qui est l’enfant de marie ?
children_of_marie = find_children_of("marie")
print("Q3. Enfant(s) de marie :", children_of_marie)
# => Attendu: ['suzanne']

# Q4. jean est-il grand-parent de thomas ?
res = "oui" if is_grandparent("jean", "thomas") else "non"
print("Q4. jean est-il grand-parent de thomas ?", res)
# => Attendu: 'non'

# Q5. Quels sont les ancêtres de thomas ?
anc_of_thomas = find_ancestors_of("thomas")
print("Q5. Ancêtres de thomas :", anc_of_thomas)
# => Attendu: ['suzanne', 'marie', 'jean']

Q1. Grand-parent(s) de thomas : ['marie']
Q2. Enfant(s) de jean : ['marie']
Q3. Enfant(s) de marie : ['suzanne']
Q4. jean est-il grand-parent de thomas ? non
Q5. Ancêtres de thomas : ['suzanne', 'marie', 'jean']
